In [ ]:

!pip -q install -U "transformers>=4.52.0,<5" "datasets>=2.20.0" evaluate accelerate
!pip -q install -U kagglehub
!pip -q install -U openai




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00


In [ ]:

import os, re, json, ast, time, random, hashlib
import numpy as np
import pandas as pd
import torch

from datasets import Dataset, DatasetDict, concatenate_datasets, ClassLabel, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
)

Chatgpt code

In [ ]:
PROMPT_VERSION = "sarcasm_v3_contextual_pragmatic"

SYSTEM = """
You are a sarcasm detector that works across cultures, dialects, and age groups.

You will receive a pair:
- Context: previous message(s)
- Reply: the response message

Label 1 (sarcasm/irony) if the reply likely conveys a meaning different from its literal wording,
often via: mock praise, contradiction, understatement/overstatement, rhetorical tone, or implied ridicule.
Use the context to infer the intended meaning.

Label 0 if the reply is likely literal (even if rude, angry, or emotional) and does not imply an opposite meaning.

Return JSON only: {"label":0 or 1, "confidence":0.0-1.0}
Confidence guide:
- 0.85-1.0: clear sarcasm/literal
- 0.60-0.84: probable but not certain
- 0.40-0.59: ambiguous, guess
"""


In [ ]:
import hashlib, json, time
import numpy as np
from openai import OpenAI
import os



client = OpenAI()

VOTE_CACHE = {}
PROMPT_VERSION = "irony_v2_strict_boundary"

def _vote_key(text: str, k: int, temperature: float, model: str) -> str:
    payload = f"{PROMPT_VERSION}||{model}||k={k}||temp={temperature}||{text.strip()}"
    return hashlib.sha256(payload.encode("utf-8")).hexdigest()

def gpt52_label_vote(text, k=3, temperature=0.7, model="gpt-5.2", max_output_tokens=80):
    key = _vote_key(text, k, temperature, model)
    if key in VOTE_CACHE:
        return VOTE_CACHE[key], True  # cached

    labels = []
    confs = []

    for _ in range(k):
        resp = client.responses.create(
            model=model,
            input=[
                {"role": "system", "content": SYSTEM},
                {"role": "user", "content": text},
            ],
            temperature=temperature,
            max_output_tokens=max_output_tokens,
        )
        out = json.loads(resp.output_text)
        labels.append(int(out["label"]))
        confs.append(float(out["confidence"]))

    # majority vote (tie -> 0, conservative)
    vote = 1 if sum(labels) > (k // 2) else 0

    result = {
        "label": vote,
        "confidence": float(np.mean(confs)),
        "votes": labels,
        "vote_rate_1": float(np.mean(labels)),  # fraction of "1" votes
    }
    VOTE_CACHE[key] = result
    return result, False


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
from tqdm.auto import tqdm

def evaluate_gpt_vote_only(ds, max_examples=None, k=3, temperature=0.7):
    texts = ds["text"]
    labels = ds["label"]

    if max_examples is not None:
        texts = texts[:max_examples]
        labels = labels[:max_examples]

    y_pred = []
    for t in tqdm(texts, desc=f"GPT vote k={k}, temp={temperature}"):
        out, _ = gpt52_label_vote(t, k=k, temperature=temperature)
        y_pred.append(int(out["label"]))

    y_true = np.array(labels, dtype=int)
    y_hat  = np.array(y_pred, dtype=int)

    f1 = f1_score(y_true, y_hat)
    acc = accuracy_score(y_true, y_hat)
    p, r, _, _ = precision_recall_fscore_support(y_true, y_hat, average="binary", zero_division=0)
    return {"n": len(y_pred), "accuracy": float(acc), "f1": float(f1), "precision": float(p), "recall": float(r)}


Load Fig dataset

In [ ]:
def figlang_label_to_y(lab: str) -> int:
    lab = (lab or "").strip().lower()

    # handle negatives FIRST (because "not_sarcasm" contains "sarc")
    if lab in ["0", "not_sarcasm", "non_sarcasm", "not sarcastic", "non sarcastic", "literal"]:
        return 0
    if lab.startswith("not") and ("sarc" in lab or "irony" in lab or "ironic" in lab):
        return 0
    if "not_sarc" in lab or "non_sarc" in lab:
        return 0

    # positives
    if lab in ["1", "sarcasm", "sarcastic", "irony", "ironic"]:
        return 1
    if "sarc" in lab or "irony" in lab or "ironic" in lab:
        return 1

    # fallback conservative
    return 0


In [ ]:
from datasets import load_dataset, Features, Value, Sequence
import random

def load_figlang2020_sarcasm_fixed(split="test", sample_n=800, seed=42):
    feats = Features({
        "label": Value("string"),
        "context": Sequence(Value("string")),
        "response": Value("string"),
        "id": Value("string"),
    })

    raw = load_dataset("tasksource/figlang2020-sarcasm", split=split, features=feats)

    def pack(ex):
        ctx = ex["context"] if ex["context"] is not None else []
        ctx = [c for c in ctx if c is not None and str(c).strip()]
        ctx_str = "\n".join([f"- {c.strip()}" for c in ctx])

        reply = (ex["response"] or "").strip()
        text = f"Context messages:\n{ctx_str}\nReply:\n{reply}"

        lab_raw = (ex["label"] or "")
        y = figlang_label_to_y(lab_raw)

        return {"text": text, "label": y, "label_raw": lab_raw}

    ds = raw.map(pack, remove_columns=raw.column_names)

    if sample_n is not None and len(ds) > sample_n:
        idx = list(range(len(ds)))
        random.Random(seed).shuffle(idx)
        ds = ds.select(idx[:sample_n])

    return ds

figlang = load_figlang2020_sarcasm_fixed(split="test", sample_n=800, seed=42)

import numpy as np, pandas as pd
print("Raw label counts:\n", pd.Series(figlang["label_raw"]).value_counts().head(10))
print("Binary label pos_rate:", float(np.mean(figlang["label"])), "n:", len(figlang))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)ection_shared_task_reddit_training.jsonl: 0.00B [00:00, ?B/s]

(…)ction_shared_task_twitter_training.jsonl: 0.00B [00:00, ?B/s]

(…)tection_shared_task_reddit_testing.jsonl: 0.00B [00:00, ?B/s]

(…)ection_shared_task_twitter_testing.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Raw label counts:
 NOT_SARCASM    403
SARCASM        397
Name: count, dtype: int64
Binary label pos_rate: 0.49625 n: 800


In [ ]:
import pandas as pd
pd.DataFrame(figlang[:10])

,label,text,label_raw
0,0,Context messages:\n- How would you feel about ...,NOT_SARCASM
1,1,Context messages:\n- @USER @USER @USER The BEA...,SARCASM
2,0,Context messages:\n- He tore up the Constituti...,NOT_SARCASM
3,1,Context messages:\n- Jeffrey Epsteins autopsy ...,SARCASM
4,0,"Context messages:\n- Although I am an atheist,...",NOT_SARCASM
5,0,Context messages:\n- Found at the entrance to ...,NOT_SARCASM
6,0,Context messages:\n- Remembered to take crib d...,NOT_SARCASM
7,1,Context messages:\n- This bipartisan statement...,SARCASM
8,0,Context messages:\n- We are meant to be with s...,NOT_SARCASM
9,1,Context messages:\n- Every time\n- Do people n...,SARCASM


Load tweet/eval

In [ ]:
external = load_dataset("cardiffnlp/tweet_eval", "irony")["train"]
# convert to your expected format
external2 = external.map(lambda x: {"text": f"Context: \nReply: {x['text']}", "label": int(x["label"])})

external2

import pandas as pd
pd.DataFrame(external[:10])




Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

,text,label
0,seeing ppl walking w/ crutches makes me really...,1
1,"look for the girl with the broken smile, ask h...",0
2,Now I remember why I buy books online @user #s...,1
3,@user @user So is he banded from wearing the c...,1
4,Just found out there are Etch A Sketch apps. ...,1
5,"Hey what do you know, one of the witnesses sup...",1
6,@user on stage at #flzjingleball at the @user ...,0
7,You know it's going to be a great day when you...,1
8,Halfway thorough my workday ... Woooo,1
9,Would like to thank my nephew for giving me hi...,1


Chatgpt eva

In [ ]:
res_figlang = evaluate_gpt_vote_only(figlang.remove_columns(["label_raw"]), k=3, temperature=0.7)
res_figlang

GPT vote k=3, temp=0.7:   0%|          | 0/800 [00:00<?, ?it/s]

{'n': 800,
 'accuracy': 0.7175,
 'f1': 0.7532751091703057,
 'precision': 0.6647398843930635,
 'recall': 0.8690176322418136}

In [ ]:
res_tweets = evaluate_gpt_vote_only(external2, k=3, temperature=0.7)
res_tweets

GPT vote k=3, temp=0.7:   0%|          | 0/784 [00:00<?, ?it/s]

{'n': 784,
 'accuracy': 0.7869897959183674,
 'f1': 0.7867177522349936,
 'precision': 0.652542372881356,
 'recall': 0.9903536977491961}

new prompt trial

In [ ]:
PROMPT_VERSION = "sarcasm_general_v4"

SYSTEM = r"""
You are a sarcasm/irony detector that generalizes across cultures, dialects, and age groups (teens/adults).
You may see explicit sarcasm markers like "/s", "s/", "#sarcasm", "#irony", "sarcasm:", "jk", "j/k", "just kidding",
eye-roll 🙄, "yeah right", scare quotes, exaggerated praise, mock politeness, contradiction with context, understatement/overstatement,
rhetorical questions, or implicit ridicule.

Task:
Given (Context messages, Reply), output whether the Reply is sarcastic/ironic.

Label policy:
- label=1 if the Reply likely conveys an implied meaning that differs from the literal wording, often mock/ironic.
- label=0 if the Reply is likely literal (even if angry, rude, emotional, or humorous) and does not rely on ironic reversal.

Use context heavily: sarcasm often depends on prior turns.
Do NOT confuse profanity, insults, or negativity with sarcasm unless there is ironic reversal/mock praise.
If explicit sarcasm markers exist (e.g., "/s"), treat as sarcasm unless it is clearly quoting or discussing markers.

Output STRICT JSON only:
{"label":0 or 1, "confidence":0.0-1.0}

Confidence guide:
- 0.90-1.00: very clear
- 0.70-0.89: likely
- 0.50-0.69: ambiguous but leaning
"""

In [ ]:
res_figlang = evaluate_gpt_vote_only(figlang.remove_columns(["label_raw"]), k=3, temperature=0.7)
res_figlang

GPT vote k=3, temp=0.7:   0%|          | 0/800 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
res_tweets = evaluate_gpt_vote_only(external2, k=3, temperature=0.7)
res_tweets

trial #2 dor res_tweets

In [ ]:
def build_input(context_text: str, reply_text: str) -> str:
    # Keep format stable (helps caching + consistent training)
    context_text = clean_text(context_text)
    reply_text = clean_text(reply_text)
    return f"Context: {context_text}\nReply: {reply_text}"

In [ ]:
# ============================================
# 12) QUOTA-SAFE GPT-5.2 ROUTER WITH CACHING (IMPROVED)
#     - Prompt calibrated to TweetEval "irony" boundary
#     - Cache versioning (prompt changes invalidate old cache)
#     - Precision-safe routing (avoid calling GPT for strong non-irony)
# ============================================

import os, time, json, hashlib
from openai import OpenAI

# ✅ DO NOT hardcode keys in notebooks. Use Colab Secrets or environment variables instead.
# In Colab: Runtime -> Secrets -> add OPENAI_API_KEY
# Or set it once in a private cell: os.environ["OPENAI_API_KEY"] = "..."
client = OpenAI()

# ---- Prompt versioning (important for cache!) ----
PROMPT_VERSION = "irony_v2_strict_boundary"

SYSTEM = (
    "You are labeling tweets for the sarcasm detection task.\n"
    "Label 1 ONLY if the intended meaning clearly CONTRASTS with the literal wording "
    "(e.g., exaggerated praise for a bad situation, saying the opposite, clear ironic twist).\n\n"
    "Label 0 if it is:\n"
    "- a literal complaint or negativity (even if rude)\n"
    "- a plain opinion with no clear contrast\n"
    "- a factual statement\n"
    "- a rhetorical question with no clear opposite-meaning\n"
    "- sarcasm/irony that is NOT clearly inferable from the text alone\n\n"
    "If unsure, choose label 0 with confidence <= 0.6.\n"
    "Return JSON only: {\"label\":0 or 1, \"confidence\":0.0-1.0}. No other text."
)

# ---- Cache + quota ----
CACHE = {}  # for persistence, replace with sqlite/redis
DAILY_CALL_LIMIT = 300
_calls_today = 0
_day_key = time.strftime("%Y-%m-%d")

def _hash_key(text: str) -> str:
    """
    Versioned cache key so prompt updates don't reuse old cached labels.
    """
    payload = f"{PROMPT_VERSION}||{text.strip()}"
    return hashlib.sha256(payload.encode("utf-8")).hexdigest()

def gpt52_label(text: str, model="gpt-5.2", max_output_tokens=40):
    global _calls_today, _day_key

    now_day = time.strftime("%Y-%m-%d")
    if now_day != _day_key:
        _day_key = now_day
        _calls_today = 0

    k = _hash_key(text)
    if k in CACHE:
        return CACHE[k], True  # cached

    if _calls_today >= DAILY_CALL_LIMIT:
        return {"label": None, "confidence": 0.0}, False  # budget stop

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": text},
        ],
        max_output_tokens=max_output_tokens,
    )

    out = json.loads(resp.output_text)
    # Basic guardrails to prevent malformed outputs from breaking pipeline
    if "label" not in out or "confidence" not in out:
        out = {"label": 0, "confidence": 0.0}

    out["label"] = int(out["label"])
    out["confidence"] = float(out["confidence"])

    CACHE[k] = out
    _calls_today += 1
    return out, False

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
from tqdm.auto import tqdm

def dataset_to_text_and_labels(ds, text_col="text", label_col="label"):
    cols = set(ds.column_names)

    if text_col in cols:
        texts = ds[text_col]
    elif "context_text" in cols and "reply_text" in cols:
        texts = [build_input(c, r) for c, r in zip(ds["context_text"], ds["reply_text"])]
    else:
        raise ValueError(
            f"Dataset must have '{text_col}' or ('context_text' and 'reply_text'). "
            f"Found columns: {ds.column_names}"
        )

    if label_col in cols:
        labels = ds[label_col]
    else:
        labels = None

    return texts, labels

In [ ]:
def evaluate_gpt_only(ds, max_examples=200):
    texts, labels = dataset_to_text_and_labels(ds)
    texts, labels = texts[:max_examples], labels[:max_examples]

    y_pred = []
    for t in texts:
        out, _ = gpt52_label(t)  # cached + limit applies
        if out["label"] is None:
            break
        y_pred.append(int(out["label"]))

    y_true = np.array(labels[:len(y_pred)], dtype=int)
    y_hat = np.array(y_pred, dtype=int)

    from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
    f1 = f1_score(y_true, y_hat)
    acc = accuracy_score(y_true, y_hat)
    p, r, _, _ = precision_recall_fscore_support(y_true, y_hat, average="binary", zero_division=0)
    return {"n": len(y_pred), "accuracy": float(acc), "f1": float(f1), "precision": float(p), "recall": float(r)}

evaluate_gpt_only(external2, max_examples=2000)

In [ ]:
import pandas as pd
pd.DataFrame(figlang.remove_columns(["label_raw"])[:10])

,label,text
0,0,Context messages:\n- How would you feel about ...
1,1,Context messages:\n- @USER @USER @USER The BEA...
2,0,Context messages:\n- He tore up the Constituti...
3,1,Context messages:\n- Jeffrey Epsteins autopsy ...
4,0,"Context messages:\n- Although I am an atheist,..."
5,0,Context messages:\n- Found at the entrance to ...
6,0,Context messages:\n- Remembered to take crib d...
7,1,Context messages:\n- This bipartisan statement...
8,0,Context messages:\n- We are meant to be with s...
9,1,Context messages:\n- Every time\n- Do people n...


In [ ]:
evaluate_gpt_only(figlang.remove_columns(["label_raw"]), max_examples=800)

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'n': 0, 'accuracy': nan, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0}